In [5]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools
from datetime import datetime

In [36]:
dataset=pd.read_csv("iris-versicolor.csv")
X = dataset.columns[1:]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)


In [ ]:
df=dataset[50:101]
df=df.append(dataset[0:50])
df=df.append(dataset[101:])
df

In [37]:
X = df.columns[1:]
X = df[X]
y = df.columns[0]
y = df[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)


In [38]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [39]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        #gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        #mse=mean_squared_error(y, y_predict, squared=False)
        acc=correct/len(y_predict)
        print("Accuracy",correct/len(y_predict))
        #print("Errore quadratico medio: ", mse)
        return acc
    
    def accuracy(self, X, y):
        y_predict=self.predict(X)
        correct = np.sum(y_predict == y)
        acc=correct/len(y_predict)
        return acc
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y, decaying_func="linear"):
        n_samples, n_features = X_train.shape
        pos = len(y[y==1])
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)

        if(decaying_func=="linear"):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(decaying_func=="exponential"):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=pos/n_samples
        r_min=1
        self.m=func[0:pos]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[pos:n_samples]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y, decaying_func="linear"):
        
        self.m_func(X_train, y, decaying_func)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))
    

In [41]:
C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
parameters = {'C': C_vals, 'sigma': sigma}
#err = []
acc = []
#metto shuffle a False, è richiesto nella classe di indicare la suddivisione nel training set di dati positivi e negativi
skf = StratifiedKFold(n_splits=5, shuffle=False)
print(datetime.now())
for train_index, validation_index in skf.split(X, y):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)
    clf = GridSearchCV(model, parameters, cv=5)
    grid_result = clf.fit(X=X_train, y=y_train)
    best_params = clf.best_params_
    best_C = best_params['C']
    best_sigma = best_params['sigma']
    print("\nBest model", best_C, best_sigma)
    best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)
    best_model.fit(X_train, y_train)
    #err.append(best_model.score(X_validation, y_validation))
    acc.append(best_model.score(X_validation, y_validation))
print(datetime.now())    

2020-10-03 18:05:21.705793
     pcost       dcost       gap    pres   dres
 0: -1.4060e+01 -4.7732e+00  5e+02  2e+01  3e-16
 1: -2.7165e+00 -1.8483e+00  3e+01  2e+00  5e-16
 2: -3.9544e-01 -1.6078e+00  2e+00  4e-02  6e-16
 3: -3.1668e-01 -4.7695e-01  2e-01  1e-16  2e-15
 4: -3.1875e-01 -3.2067e-01  2e-03  2e-16  3e-16
 5: -3.1889e-01 -3.1897e-01  7e-05  2e-16  3e-16
 6: -3.1890e-01 -3.1890e-01  3e-06  3e-17  2e-16
 7: -3.1890e-01 -3.1890e-01  5e-08  1e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4060e+01 -4.7732e+00  5e+02  2e+01  3e-16
 1: -2.7165e+00 -1.8483e+00  3e+01  2e+00  5e-16
 2: -3.9544e-01 -1.6078e+00  2e+00  4e-02  6e-16
 3: -3.1668e-01 -4.7695e-01  2e-01  1e-16  2e-15
 4: -3.1875e-01 -3.2067e-01  2e-03  2e-16  3e-16
 5: -3.1889e-01 -3.1897e-01  7e-05  2e-16  3e-16
 6: -3.1890e-01 -3.1890e-01  3e-06  3e-17  2e-16
 7: -3.1890e-01 -3.1890e-01  5e-08  1e-16  2e-16
Optimal solution found.
0 out of 24 predictions correct
Accuracy 0.0


     pcost       dcost       gap    pres   dres
 0: -8.8207e+00 -3.8640e+00  5e+02  2e+01  5e-16
 1: -2.7965e+00 -2.9147e+00  3e+01  1e+00  6e-16
 2: -8.8094e-01 -1.9755e+00  2e+00  5e-02  2e-15
 3: -7.3291e-01 -9.4131e-01  2e-01  2e-16  1e-15
 4: -7.4711e-01 -7.7793e-01  3e-02  2e-16  6e-16
 5: -7.5290e-01 -7.5788e-01  5e-03  2e-16  4e-16
 6: -7.5424e-01 -7.5539e-01  1e-03  2e-17  3e-16
 7: -7.5456e-01 -7.5493e-01  4e-04  7e-17  4e-16
 8: -7.5470e-01 -7.5476e-01  7e-05  6e-17  5e-16
 9: -7.5472e-01 -7.5473e-01  1e-05  9e-17  5e-16
10: -7.5473e-01 -7.5473e-01  2e-07  1e-16  6e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.8207e+00 -3.8640e+00  5e+02  2e+01  5e-16
 1: -2.7965e+00 -2.9147e+00  3e+01  1e+00  6e-16
 2: -8.8094e-01 -1.9755e+00  2e+00  5e-02  2e-15
 3: -7.3291e-01 -9.4131e-01  2e-01  2e-16  1e-15
 4: -7.4711e-01 -7.7793e-01  3e-02  2e-16  6e-16
 5: -7.5290e-01 -7.5788e-01  5e-03  2e-16  4e-16
 6: -7.5424e-01 -7.5539e-01  1e-03  2e-17  3e-1

     pcost       dcost       gap    pres   dres
 0: -2.6926e+01 -2.0301e+00  6e+02  3e+01  2e-15
 1: -6.5525e-01 -2.0142e+00  9e+00  3e-01  2e-15
 2: -3.4140e-01 -1.2919e+00  1e+00  1e-02  6e-16
 3: -3.1956e-01 -3.5180e-01  3e-02  4e-05  2e-15
 4: -3.1999e-01 -3.2032e-01  3e-04  4e-07  4e-16
 5: -3.2000e-01 -3.2001e-01  6e-06  7e-09  4e-16
 6: -3.2000e-01 -3.2000e-01  2e-06  2e-09  4e-16
 7: -3.2000e-01 -3.2000e-01  1e-06  7e-10  4e-16
 8: -3.2000e-01 -3.2000e-01  4e-07  2e-10  3e-16
 9: -3.2000e-01 -3.2000e-01  8e-08  3e-11  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6926e+01 -2.0301e+00  6e+02  3e+01  2e-15
 1: -6.5525e-01 -2.0142e+00  9e+00  3e-01  2e-15
 2: -3.4140e-01 -1.2919e+00  1e+00  1e-02  6e-16
 3: -3.1956e-01 -3.5180e-01  3e-02  4e-05  2e-15
 4: -3.1999e-01 -3.2032e-01  3e-04  4e-07  4e-16
 5: -3.2000e-01 -3.2001e-01  6e-06  7e-09  4e-16
 6: -3.2000e-01 -3.2000e-01  2e-06  2e-09  4e-16
 7: -3.2000e-01 -3.2000e-01  1e-06  7e-10  4e-1

     pcost       dcost       gap    pres   dres
 0: -4.7132e+01 -1.9877e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00  5e+00  2e-01  3e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  2e-16  5e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  4e-17  6e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  8e-17  5e-16
 5: -8.0000e-01 -8.0001e-01  6e-06  2e-16  1e-15
 6: -8.0000e-01 -8.0000e-01  4e-07  2e-16  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7132e+01 -1.9877e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00  5e+00  2e-01  3e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  2e-16  5e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  4e-17  6e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  8e-17  5e-16
 5: -8.0000e-01 -8.0001e-01  6e-06  2e-16  1e-15
 6: -8.0000e-01 -8.0000e-01  4e-07  2e-16  5e-16
Optimal solution found.
24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -2.6933e+01 -2.0277e+00  6e+02  3e+01  2e-15
 1: -6.5528e-01 -2.0141e+00

     pcost       dcost       gap    pres   dres
 0: -4.7133e+01 -1.9873e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00  5e+00  2e-01  4e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  4e-17  6e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  3e-16  5e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  4e-16  5e-16
 5: -8.0000e-01 -8.0001e-01  6e-06  2e-16  9e-16
 6: -8.0000e-01 -8.0000e-01  6e-08  7e-16  1e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.7133e+01 -1.9873e+00  4e+02  2e+01  3e-15
 1: -1.1495e+00 -1.9791e+00  5e+00  2e-01  4e-15
 2: -7.5209e-01 -1.4188e+00  7e-01  4e-17  6e-16
 3: -7.9952e-01 -8.5621e-01  6e-02  3e-16  5e-16
 4: -7.9999e-01 -8.0056e-01  6e-04  4e-16  5e-16
 5: -8.0000e-01 -8.0001e-01  6e-06  2e-16  9e-16
 6: -8.0000e-01 -8.0000e-01  6e-08  7e-16  1e-15
Optimal solution found.
24 out of 24 predictions correct
Accuracy 1.0
     pcost       dcost       gap    pres   dres
 0: -1.5011e+01 -2.0063e+01  5e+02  2e+01  3e-16
 1: -5.3251e+00 -1.6512e+01

...

     pcost       dcost       gap    pres   dres
 0:  1.8241e+08 -1.0708e+09  2e+09  4e-02  4e-12
 1:  6.2385e+07 -1.3390e+08  2e+08  4e-03  1e-10
 2:  1.1872e+07 -1.8954e+07  3e+07  8e-05  6e-12
 3:  1.7515e+06 -2.1029e+06  4e+06  3e-13  1e-12
 4:  2.5177e+05 -2.8635e+05  5e+05  3e-12  5e-13
 5:  3.5874e+04 -4.0718e+04  8e+04  2e-13  2e-13
 6:  5.0363e+03 -5.9128e+03  1e+04  2e-13  6e-14
 7:  6.7750e+02 -8.8823e+02  2e+03  2e-15  2e-14
 8:  7.5974e+01 -1.4593e+02  2e+02  8e-15  8e-15
 9:  3.0427e+00 -4.6736e+01  5e+01  1e-14  4e-15
10: -1.3931e+01 -2.6097e+01  1e+01  1e-14  2e-15
11: -1.8923e+01 -2.0567e+01  2e+00  2e-14  2e-15
12: -1.9416e+01 -1.9856e+01  4e-01  8e-15  2e-15
13: -1.9547e+01 -1.9694e+01  1e-01  1e-14  2e-15
14: -1.9596e+01 -1.9675e+01  8e-02  1e-14  2e-15
15: -1.9621e+01 -1.9628e+01  6e-03  7e-16  2e-15
16: -1.9624e+01 -1.9624e+01  8e-05  2e-14  2e-15
17: -1.9624e+01 -1.9624e+01  8e-07  1e-14  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dre

     pcost       dcost       gap    pres   dres
 0:  3.2410e+08 -2.1842e+09  3e+09  6e-11  2e-11
 1:  5.5417e+07 -2.1515e+08  3e+08  1e-11  2e-11
 2:  1.9189e+07 -4.4439e+07  6e+07  4e-11  1e-11
 3:  1.2926e+07 -4.6334e+07  6e+07  6e-11  1e-11
 4:  1.1746e+07 -4.0942e+07  5e+07  2e-11  9e-12
 5:  8.0637e+06 -3.1915e+07  4e+07  7e-12  9e-12
 6:  3.4747e+06 -1.4248e+07  2e+07  3e-11  8e-12
 7:  9.2380e+05 -4.4823e+06  5e+06  1e-11  7e-12
 8:  3.6865e+05 -1.5951e+06  2e+06  9e-12  5e-12
 9:  8.1951e+04 -3.0841e+05  4e+05  7e-12  4e-12
10:  1.4886e+04 -8.8791e+04  1e+05  4e-12  2e-12
11:  1.9684e+01 -2.2116e+04  2e+04  2e-12  1e-12
12: -3.4104e+03 -2.0264e+04  2e+04  3e-12  1e-12
13: -5.9412e+03 -1.3119e+04  7e+03  9e-13  1e-12
14: -7.5913e+03 -9.6346e+03  2e+03  2e-12  2e-12
15: -8.1242e+03 -8.8010e+03  7e+02  2e-12  1e-12
16: -8.3516e+03 -8.4709e+03  1e+02  2e-12  2e-12
17: -8.4050e+03 -8.4063e+03  1e+00  1e-13  2e-12
18: -8.4056e+03 -8.4056e+03  1e-02  3e-13  2e-12
19: -8.4056e+03 -8.40

     pcost       dcost       gap    pres   dres
 0:  2.3948e+07 -7.2644e+08  8e+08  2e-11  4e-11
 1:  1.7607e+07 -8.4537e+07  1e+08  5e-12  3e-11
 2:  5.7045e+06 -3.7321e+07  4e+07  1e-11  3e-11
 3:  8.3715e+05 -4.7062e+06  6e+06  7e-11  3e-11
 4:  1.7372e+04 -1.4928e+06  2e+06  7e-12  2e-11
 5: -1.5330e+05 -4.7428e+05  3e+05  2e-11  2e-11
 6: -1.7644e+05 -3.1603e+05  1e+05  5e-11  2e-11
 7: -1.8536e+05 -2.9807e+05  1e+05  4e-11  1e-11
 8: -1.9249e+05 -2.3749e+05  5e+04  2e-11  2e-11
 9: -1.9527e+05 -2.3573e+05  4e+04  6e-11  2e-11
10: -2.0192e+05 -2.1478e+05  1e+04  2e-11  2e-11
11: -2.0514e+05 -2.0979e+05  5e+03  1e-11  1e-11
12: -2.0664e+05 -2.0779e+05  1e+03  4e-11  2e-11
13: -2.0709e+05 -2.0718e+05  9e+01  1e-11  2e-11
14: -2.0712e+05 -2.0713e+05  3e+00  2e-12  2e-11
15: -2.0712e+05 -2.0712e+05  9e-02  5e-11  2e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.3948e+07 -7.2644e+08  8e+08  2e-11  4e-11
 1:  1.7607e+07 -8.4537e+07  1e+08  5e-12  3e-1

     pcost       dcost       gap    pres   dres
 0: -2.6644e+05 -1.0687e+09  1e+09  1e-12  2e-11
 1: -2.6647e+05 -1.1005e+07  1e+07  6e-11  2e-11
 2: -2.6899e+05 -4.2839e+05  2e+05  8e-11  3e-11
 3: -3.1439e+05 -3.2298e+05  9e+03  1e-10  3e-11
 4: -3.1989e+05 -3.2102e+05  1e+03  9e-11  3e-11
 5: -3.1997e+05 -3.2018e+05  2e+02  5e-11  3e-11
 6: -3.1999e+05 -3.2009e+05  1e+02  2e-10  3e-11
 7: -3.2000e+05 -3.2003e+05  3e+01  1e-11  3e-11
 8: -3.2000e+05 -3.2001e+05  1e+01  6e-11  4e-11
 9: -3.2000e+05 -3.2000e+05  2e+00  2e-12  3e-11
10: -3.2000e+05 -3.2000e+05  2e-01  9e-13  4e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6644e+05 -1.0687e+09  1e+09  1e-12  2e-11
 1: -2.6647e+05 -1.1005e+07  1e+07  6e-11  2e-11
 2: -2.6899e+05 -4.2839e+05  2e+05  8e-11  3e-11
 3: -3.1439e+05 -3.2298e+05  9e+03  1e-10  3e-11
 4: -3.1989e+05 -3.2102e+05  1e+03  9e-11  3e-11
 5: -3.1997e+05 -3.2018e+05  2e+02  5e-11  3e-11
 6: -3.1999e+05 -3.2009e+05  1e+02  2e-10  3e-1

     pcost       dcost       gap    pres   dres
 0: -4.6671e+05 -6.6865e+08  7e+08  8e-12  3e-11
 1: -4.6717e+05 -7.4788e+06  7e+06  1e-10  3e-11
 2: -5.0805e+05 -8.6737e+05  4e+05  9e-11  4e-11
 3: -7.9707e+05 -8.2639e+05  3e+04  5e-11  5e-11
 4: -7.9996e+05 -8.0033e+05  4e+02  1e-10  7e-11
 5: -7.9999e+05 -8.0003e+05  4e+01  7e-11  6e-11
 6: -8.0000e+05 -8.0000e+05  5e+00  5e-12  6e-11
 7: -8.0000e+05 -8.0000e+05  2e+00  4e-11  6e-11
 8: -8.0000e+05 -8.0000e+05  4e-01  8e-11  7e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.6671e+05 -6.6865e+08  7e+08  8e-12  3e-11
 1: -4.6717e+05 -7.4788e+06  7e+06  1e-10  3e-11
 2: -5.0805e+05 -8.6737e+05  4e+05  9e-11  4e-11
 3: -7.9707e+05 -8.2639e+05  3e+04  5e-11  5e-11
 4: -7.9996e+05 -8.0033e+05  4e+02  1e-10  7e-11
 5: -7.9999e+05 -8.0003e+05  4e+01  7e-11  6e-11
 6: -8.0000e+05 -8.0000e+05  5e+00  5e-12  6e-11
 7: -8.0000e+05 -8.0000e+05  2e+00  4e-11  6e-11
 8: -8.0000e+05 -8.0000e+05  4e-01  8e-11  7e-1

In [42]:
print(np.mean(acc))

0.9400000000000001


In [43]:
print(acc)

[1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 0.9]
